In [ ]:
"""
Created on Fri Jun 4 15:50 2020

This is a script to convert the NEMO temperature and salinity to potential temperature and practical salinity

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import gsw
import matplotlib.pyplot as plt
import basal_melt_param.useful_functions as uf
import basal_melt_param.T_S_profile_functions as tspf
import basal_melt_param.melt_functions as meltf

import itertools

import distributed
import glob

In [ ]:
client = distributed.Client(n_workers=4, dashboard_address=':8795', local_directory='/tmp', memory_limit='6GB')

In [ ]:
client

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
nemo_run = 'bf663' # 'bf663','bi646' 

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/SMITH_'+nemo_run+'/'
inputpath_profiles='/bettik/burgardc/DATA/NN_PARAM/interim/T_S_PROF/SMITH_'+nemo_run+'/'
inputpath_isf='/bettik/burgardc/DATA/NN_PARAM/interim/ANTARCTICA_IS_MASKS/SMITH_'+nemo_run+'/'

# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
file_mask = xr.open_dataset(inputpath_data+'mask_variables_of_interest_allyy_Ant_stereo.nc')#, chunks={'x': 600, 'y': 600})
file_mask = file_mask.assign_coords({'time': range(len(file_mask.time))})#.chunk({'time': 1})
file_mask_71 = file_mask.sel(time=range(72))
file_mask2 = xr.open_dataset(inputpath_data+'mask_depth_coord_Ant_stereo.nc')


In [ ]:
isf_files = list(sorted(glob.glob(inputpath_isf+'nemo_5km_isf_masks_and_info_and_distance_oneFRIS_yy*.nc')))
file_isf_allyy = xr.open_mfdataset(isf_files, concat_dim='time', combine='nested', chunks={'x': 533, 'y': 533})
file_isf_allyy = file_isf_allyy.assign_coords({'time': range(len(file_mask_71.time))}).chunk({'time': 1})

In [ ]:
file_isf_allyy = file_isf_allyy.assign_coords({'time': range(len(file_mask_71.time))}).chunk({'time': 1})

In [ ]:
file_mask_cutted = uf.cut_domain_stereo(file_mask_71, map_lim, map_lim)#.squeeze().drop('time')
file_isf_cutted = uf.cut_domain_stereo(file_isf_allyy, map_lim, map_lim)#.squeeze().drop('time')

#file_TS_orig  = xr.open_mfdataset(inputpath_data+'variables_of_interest_2000_Ant_stereo.nc', chunks={'x': 600, 'y': 600})
#file_TS_orig_cutted = uf.cut_domain_stereo(file_TS_orig, map_lim, map_lim).squeeze().drop('time')

In [ ]:
lon = file_isf_cutted['longitude']
lat = file_isf_cutted['latitude']

In [ ]:
ds_ts  = xr.open_dataset(inputpath_data + '3D_variables_of_interest_allyy_Ant_stereo.nc') #, chunks={'x': 600, 'y': 600})
ds_ts = ds_ts.assign_coords({'time': range(len(ds_ts.time))}).chunk({'time': 1})
ds_ts_71 = ds_ts.sel(time=range(72))
ds_ts_71 = ds_ts_71[['thetao', 'so']]
ds_ts_cutted = uf.cut_domain_stereo(ds_ts, map_lim, map_lim)

Prepare the depth axis

In [ ]:
nemo_depth = np.round(file_mask2['gdept_0'].squeeze(dim=['lon','lat']), 3) # round to mm scale - should be enough

Cut out the temperature and salinity and assign the new depth axis

In [ ]:
ds_temp_saline_input = ds_ts_cutted[['thetao', 'so']]
ds_temp_saline_input = ds_temp_saline_input.rename({'thetao': 'temperature', 'so': 'salinity'})
ds_temp_saline_input = ds_temp_saline_input.rename({'deptht': 'depth'})
ds_temp_saline_input['depth'] = np.round(ds_temp_saline_input.depth, 3)
ds_temp_saline_input = ds_temp_saline_input.assign_coords(depth=nemo_depth.values)


In [ ]:
## only points where there is no bedrock - I think not needed for NEMO data
#vert_mask = file_in_T['z_bnds'][:,1] > interp_bed['bed'] 
# only points of open ocean
mask_ocean = file_isf_cutted['ISF_mask'] == 1  #ocean without ice shelf cavity

CONVERT CONSERVATIVE TEMPERATURE FOR OPEN OCEAN REGIONS TO POTENTIAL TEMPERATURE 
AND ABSOLUTE SALIINITY TO PRACTICAL SALINITY

In [ ]:
ds_ts_cutted.time

In [ ]:
for tt,timet in enumerate(ds_ts_cutted.time.sel(time=range(34,41))):
    ds_temp_saline_input_yy = ds_temp_saline_input.sel(time=timet).load()
    mask_ocean_yy = mask_ocean.sel(time=timet).load()
    ds_temp_saline_input_yy['theta_ocean'] = ds_temp_saline_input_yy['temperature'].where(mask_ocean_yy)
    ds_temp_saline_input_yy['salinity_ocean'] = xr.apply_ufunc(gsw.SP_from_SA, ds_temp_saline_input_yy['salinity'].where(mask_ocean_yy), ds_temp_saline_input_yy['depth'], lon, lat, dask = 'allowed')
    ds_temp_saline_output = ds_temp_saline_input_yy[['theta_ocean', 'salinity_ocean']]
    ds_temp_saline_output.to_netcdf(inputpath_profiles + 'T_S_theta_ocean_corrected_yy'+str(timet.values).zfill(2)+'.nc')

Write the results to multiple files (1 per year)

In [ ]:
yearly_datasets = list(tspf.split_by_chunks(ds_temp_saline_output.unify_chunks(),'time'))
paths = [tspf.create_filepath(ds, 'T_S_theta_ocean_corrected', inputpath_profiles, ds.time[0].values) for ds in yearly_datasets]

this takes approximately 1 min per year

In [ ]:
xr.save_mfdataset(datasets=yearly_datasets, paths=paths)